# School Enrollment Rates

School Enrollment Rates


In [24]:
from ambry import get_library
import censuslib.dataframe
l = get_library()
b = l.bundle('census.gov-acs-enrollment-p5ye2014-hdp-0.0.1')

def ca_county_pred(r):
    from geoid.civick import GVid
    return (r.stusab == 'ca' and GVid.parse(r.gvid, exception=False).level == 'county')


enroll = l.partition('census.gov-acs-p5ye2014-b14001').analysis.dataframe(ca_county_pred, 
                                                                          df_class=censuslib.dataframe.CensusDataFrame)

enroll.set_index('gvid',inplace=True)

In [33]:
# Population under 18
u18 = l.partition('census.gov-acs-p5ye2014-b09001').analysis.dataframe(ca_county_pred, 
                                                                          df_class=censuslib.dataframe.CensusDataFrame)

u18.set_index('gvid',inplace=True)

In [34]:
def ca_county_pred_fp(r):
    from geoid.civick import GVid
    return (r.statefp == 6 and GVid.parse(r.gvid, exception=False).level == 'county')


counties = l.partition('census.gov-tiger-2015-counties').analysis.dataframe(ca_county_pred_fp, 
                                                                          df_class=censuslib.dataframe.CensusDataFrame)
counties.set_index('gvid',inplace=True)

In [41]:
# Create an under 5 category for the U18 df so we can compare it to the under5 category from sex by age. 
u18.add_sum_m('under5',3,4)
u18['under3'] = u18['b09001003']


In [79]:
df = counties[['name']].copy()
df['state'] = 'CA'
df['enrolled_prek'],df['enrolled_prek_m90']  = enroll['b14001003'], enroll['b14001003_m90']
df['enrolled_k'],df['enrolled_k_m90']  = enroll['b14001004'],enroll['b14001004_m90']
df.add_sum_m('enrolled','enrolled_prek', 'enrolled_k')
df['3to4'] = u18['b09001004']
df['3to4_m90'] = u18['b09001004_m90']
df['not_enrolled_prek'] = df['3to4'] - df['enrolled_prek']
df['enrolled_rate'], df['enrolled_rate_m90']  = df.ratio('enrolled_prek', '3to4')
df['enrolled_rate_rse'] = df['enrolled_rate_m90'] / 1.645 / df['enrolled_rate'] * 100
df['not_enrolled_rate'] = 1.0 - df['enrolled_rate']
df.columns = ['county'] + list(df.columns)[1:]
not_enrolled = df[['state','county','enrolled_prek','not_enrolled_prek','enrolled_rate', 'not_enrolled_rate', 'enrolled_rate_m90','enrolled_rate_rse']] \
    .sort('not_enrolled_rate', ascending=False)
not_enrolled

,state,county,enrolled_prek,not_enrolled_prek,enrolled_rate,not_enrolled_rate,enrolled_rate_m90,enrolled_rate_rse
gvid,,,,,,,,
0O060l,CA,Glenn,291,597,0.328,0.672,0.132158,24.493642
0O060D,CA,Madera,1747,2868,0.379,0.621,0.050254,8.060515
0O060v,CA,Kings,1946,3156,0.381,0.619,0.054804,8.744281
0O061t,CA,Sierra,20,32,0.385,0.615,0.290122,45.809312
0O060t,CA,Kern,12149,17974,0.403,0.597,0.022657,3.417710
0O060L,CA,Merced,3717,5281,0.413,0.587,0.039701,5.843721
0O0611,CA,Plumas,148,209,0.415,0.585,0.194214,28.448897
0O060j,CA,Fresno,14409,19705,0.422,0.578,0.021229,3.058108
0O061J,CA,Tulare,7531,10100,0.427,0.573,0.033385,4.752952


In [80]:
not_enrolled.to_csv('not_enrolled_prek.csv')

In [64]:
!pwd

/Users/eric/proj/virt/ambry-develop/ambry10-bundles/sandag.org/crime-orig


In [65]:
len(not_enrolled)


54